In [3]:
from transformers import CamembertTokenizer,CamembertModel
import torch 
import torch.nn as nn
import torch.nn.functional as F

from src.model import Full_Network

## Load Tokenizer 

In [4]:
tokenizer = CamembertTokenizer.from_pretrained('camembert-base',
                                               sep_token='</s>', 
                                               cls_token='<s>', 
                                               do_lower_case=True
                                                )

## Load Model and parameters 

In [5]:
model = Full_Network()    
model.load_state_dict(torch.load('./models/model_train_full_finetune.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [30]:

question = 'Comment s\'appelle le portail open data du gouvernement ?'
positive = 'Etalab est une administration publique française qui fait notamment office de Chief Data Officer de l\'État et coordonne la conception et la mise en œuvre de sa stratégie dans le domaine de la donnée (ouverture et partage des données publiques ou open data, exploitation des données et intelligence artificielle...). Ainsi, Etalab développe et maintient le portail des données ouvertes du gouvernement français data.gouv.fr. Etalab promeut également une plus grande ouverture l\'administration sur la société (gouvernement ouvert) : transparence de l\'action publique, innovation ouverte, participation citoyenne... elle promeut l’innovation, l’expérimentation, les méthodes de travail ouvertes, agiles et itératives, ainsi que les synergies avec la société civile pour décloisonner l’administration et favoriser l’adoption des meilleures pratiques professionnelles dans le domaine du numérique. À ce titre elle étudie notamment l’opportunité de recourir à des technologies en voie de maturation issues du monde de la recherche. Cette entité chargée de l\'innovation au sein de l\'administration doit contribuer à l\'amélioration du service public grâce au numérique. Elle est rattachée à la Direction interministérielle du numérique, dont les missions et l’organisation ont été fixées par le décret du 30 octobre 2019.  Dirigé par Laure Lucchesi depuis 2016, elle rassemble une équipe pluridisciplinaire d\'une trentaine de personnes.'
negative = 'Les colons américains, en particulier les marchands des ports de la Nouvelle-Angleterre, reprochaient à la Grande-Bretagne sa politique commerciale : le trafic de certaines marchandises comme le thé était réservé aux navires britanniques, en vertu du monopole en vigueur. D\'autre part, dans le but d\'atrophier l\'économie américaine, les Britanniques en vinrent à interdire à leurs colonies de vendre leurs produits à un autre pays que la Grande-Bretagne, car l\'on estimait que si les colons avaient le droit de vendre leurs produits comme bon leur semblait et à qui bon leur semblait, les treize colonies américaines regorgeraient d\'argent, argent qui ne profiterait pas à la couronne.'


## Test the model

In [44]:
question'

"Comment s'appelle le portail open data du gouvernement ?"

### positive example

In [50]:
positive

"Etalab est une administration publique française qui fait notamment office de Chief Data Officer de l'État et coordonne la conception et la mise en œuvre de sa stratégie dans le domaine de la donnée (ouverture et partage des données publiques ou open data, exploitation des données et intelligence artificielle...). Ainsi, Etalab développe et maintient le portail des données ouvertes du gouvernement français data.gouv.fr. Etalab promeut également une plus grande ouverture l'administration sur la société (gouvernement ouvert) : transparence de l'action publique, innovation ouverte, participation citoyenne... elle promeut l’innovation, l’expérimentation, les méthodes de travail ouvertes, agiles et itératives, ainsi que les synergies avec la société civile pour décloisonner l’administration et favoriser l’adoption des meilleures pratiques professionnelles dans le domaine du numérique. À ce titre elle étudie notamment l’opportunité de recourir à des technologies en voie de maturation issues

In [55]:
#encode inout
encoded_query = tokenizer.encode_plus(question,
                                      positive,
                                      add_special_tokens=True,
                                      padding='max_length',
                                      truncation=True, 
                                      return_attention_mask=True,
                                      return_tensors="pt"
                                      )

In [56]:
#pass through the model

with torch.no_grad() :
    score = model(encoded_query['input_ids'],encoded_query['attention_mask'])

In [59]:
print('The relevance score of the positive example is : {}'.format(score.numpy()[0][0]))

The relevance score of the positive example is : 0.9999905824661255


### Negative example

In [52]:
negative

"Les colons américains, en particulier les marchands des ports de la Nouvelle-Angleterre, reprochaient à la Grande-Bretagne sa politique commerciale : le trafic de certaines marchandises comme le thé était réservé aux navires britanniques, en vertu du monopole en vigueur. D'autre part, dans le but d'atrophier l'économie américaine, les Britanniques en vinrent à interdire à leurs colonies de vendre leurs produits à un autre pays que la Grande-Bretagne, car l'on estimait que si les colons avaient le droit de vendre leurs produits comme bon leur semblait et à qui bon leur semblait, les treize colonies américaines regorgeraient d'argent, argent qui ne profiterait pas à la couronne."

In [60]:
#encode input
encoded_query = tokenizer.encode_plus(question,
                                      negative,
                                      add_special_tokens=True,
                                      padding='max_length',
                                      truncation=True, 
                                      return_attention_mask=True,
                                      return_tensors="pt"
                                      )

In [61]:
with torch.no_grad() :
    score = model(encoded_query['input_ids'],encoded_query['attention_mask'])

In [62]:
print('The relevance score of the positive example is : {}'.format(score.numpy()[0][0]))

The relevance score of the positive example is : 6.953609408810735e-05
